# Desafio 1: Properti

## Objetivos:
-  Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.

-  Realizar un análisis descriptivo de las principales variables.

- Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

## Preparacion de ambiente
### Importar Librerias
**Numpy** y **Pandas** son importadas para administrar todas las operaciones analiticas 

**IPython** es utilizados para administrar la visualizacion de los datos

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

### Importar archivo CSV
Utilizando Pandas importamos el archivo CSV considerando la comma como delimitador 

In [2]:
dfprop=pd.read_csv(filepath_or_buffer='properatti.csv',delimiter=',',index_col = 0 )

## Visualizacion
### Visualizar la forma del Dataframe
Se utilizara la funcion nativa **shape** de la clase DataFrame para visualizar:
- Cantidad de observaciones 
- Cantidad de variables

In [3]:
dfprop.shape

(121220, 25)

### Visualizar estructura del Dataframe
Se utilizara la funcion nativa **info()** de la clase DataFrame para visualizar:
- Cantidad de variables
- Nombre de variables
- Cantidad de registros con valores **No nulos**
- Tipo de dato

In [4]:
dfprop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121220 entries, 0 to 121219
Data columns (total 25 columns):
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 n

## Visualizacion de valores Nan en cada variable

Visualizamos la cantidad de valores Nan para cada variable  

In [5]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

## Limpieza de datos
### Eliminacion de inmuebles sin valor analitico

Eliminamos los inmuebles sin precio ni ubicaciones, ya que no tenemos ninguna forma de estimar un precio.

Condicion:
- price == Nan
- price_aprox_local_currency == Nan
- price_aprox_usd == Nan
- surface_total_in_m2 == Nan
- surface_covered_in_m2 == Nan


Los registros eliminados son 4733 

In [6]:
# Generamos una mascara para encontrar todos los registros bajo esta condicion
dfprop_withoutprice_withoutsurface=((dfprop['price'].isna()==True)\
                                    &(dfprop['price_aprox_local_currency'].isna()==True)\
                                    &(dfprop['price_aprox_usd'].isna()==True)\
                                    &(dfprop['surface_total_in_m2'].isna()==True)\
                                    &(dfprop['surface_covered_in_m2'].isna()==True))

# Instanciamos el Dataframe excluyendo los registros que no cumple con dicha condicion
dfprop=dfprop.loc[~dfprop_withoutprice_withoutsurface]


Visualizamos la forma para confirmar que se hayan eliminado los registros

In [7]:
dfprop.shape

(116487, 25)

### Separar en un nuevo Dataframe los inmuebles cuya concentracion por provincia sea menor a 30 operaciones 

Dejaremos en un nuevo DataFrame denominado 'dfprop_fewlocations' las propiedades cuya agrupacion por provincia sea menor a 30 por no tener un valor referencial para la estimacion de precios

In [8]:
#Creamos dataframe con un valor booleano que indica si la provincia tiene menos de 30 propiedades (True) o no (False).
dfprop_stateless30locations=dfprop.groupby('state_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_stateless30locations.rename(columns={'state_name':'state_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_stateless30locations,how='left',on='state_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a provincias con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(74, 25)

In [9]:
#Corroboramos que se hayan eliminado las propiedades del dataframe principal
dfprop.shape

(116413, 25)

Eliminamos propiedad en moneda distinta a ARS o USD (se excluye 'NaN' siendo que tendran tratamiento mas adelante) 

In [10]:
dfprop['currency'].unique()


array(['USD', nan, 'ARS', 'PEN', 'UYU'], dtype=object)

In [11]:
dfprop=dfprop.loc[((dfprop['currency']!='UYU')&(dfprop['currency']!='PEN'))]

#Corroboramos que se hayan eliminado las propiedades del dataframe principal
dfprop['currency'].unique()

array(['USD', nan, 'ARS'], dtype=object)

Eliminamos la categoria 'store', siendo que nuestro analisis será para viviendas de ocupacion permanente.

In [12]:
dfprop['property_type'].unique()

array(['PH', 'apartment', 'house', 'store'], dtype=object)

In [13]:
dfprop=dfprop.loc[(dfprop['property_type']!='store')]

In [14]:
dfprop['property_type'].unique()

array(['PH', 'apartment', 'house'], dtype=object)

# Imputacion
## Variable place_name - valores Nan:  
- Analizamos los valores NaN de la columna 'place_name':

In [15]:
dfprop.loc[(dfprop['place_name'].isna() == True),['place_name','place_with_parent_names']]

,place_name,place_with_parent_names
6201,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
9686,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
10855,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
14066,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
17672,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
20765,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
22398,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
23404,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
36842,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
43577,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||


- Vemos que, de acuerdo a 'place_name_with_parents', todos los valores faltantes son 'Tigre'.
- Por lo tanto, procedemos a reemplazarlos:

In [16]:
dfprop['place_name'].fillna( value = 'Tigre', inplace = True)

Volvemos a visualizar la cantidad de valores nulos para confirmar que no haya mas valores **Nan** en la variable 'place_name'

In [17]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    17992
lat-lon                        48544
lat                            48544
lon                            48544
price                          14973
currency                       14974
price_aprox_local_currency     14973
price_aprox_usd                14973
surface_total_in_m2            33301
surface_covered_in_m2          14356
price_usd_per_m2               45984
price_per_m2                   27407
floor                         104901
rooms                          67543
expenses                       98647
properati_url                      0
description                        2
title                              0
image_thumbnail                 2666
dtype: int64

## Variables 'surface_total_in_m2' y 'surface_covered_in_m2' - valores Nan
Analizamos los valores que tienen 'surface_covered_in_m2', pero no 'surface_total_in_m2'

In [18]:
dfprop.isnull().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    17992
lat-lon                        48544
lat                            48544
lon                            48544
price                          14973
currency                       14974
price_aprox_local_currency     14973
price_aprox_usd                14973
surface_total_in_m2            33301
surface_covered_in_m2          14356
price_usd_per_m2               45984
price_per_m2                   27407
floor                         104901
rooms                          67543
expenses                       98647
properati_url                      0
description                        2
title                              0
image_thumbnail                 2666
dtype: int64

In [19]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_total_in_m2'].isna() == True) & (dfprop['surface_covered_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].sample(10)

,surface_total_in_m2,surface_covered_in_m2
34076,NaN,24.0
10806,NaN,20.0
35546,NaN,65.0
18680,NaN,24.0
78580,NaN,315.0
73942,NaN,37.0
59533,NaN,180.0
41806,NaN,43.0
924,NaN,140.0
42610,NaN,120.0


Reemplazamos los valores NaN de la columna 'surface_total_in_m2', asumiendo que en estos casos es igual al valor de 'surface_covered_in_m2'

In [20]:
dfprop.fillna({'surface_total_in_m2' : dfprop['surface_covered_in_m2']}, inplace = True)

Volvemos a constatar la cantidad de valores nulos en el dataframe:

In [21]:
dfprop.isnull().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    17992
lat-lon                        48544
lat                            48544
lon                            48544
price                          14973
currency                       14974
price_aprox_local_currency     14973
price_aprox_usd                14973
surface_total_in_m2             7283
surface_covered_in_m2          14356
price_usd_per_m2               45984
price_per_m2                   27407
floor                         104901
rooms                          67543
expenses                       98647
properati_url                      0
description                        2
title                              0
image_thumbnail                 2666
dtype: int64

Repetimos la operación para los casos en los que no tenemos valor de 'surface_covered_in_m2', completando con el valor de 'surface_total_in_m2':

In [22]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_covered_in_m2'].isna() == True) & (dfprop['surface_total_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].sample(10)

,surface_total_in_m2,surface_covered_in_m2
34121,360.0,NaN
115457,0.0,NaN
99338,550.0,NaN
68103,50.0,NaN
102731,40.0,NaN
19859,629.0,NaN
50539,52.0,NaN
59711,48.0,NaN
99522,76.0,NaN
31458,42.0,NaN


In [23]:
dfprop.fillna({'surface_covered_in_m2' : dfprop['surface_total_in_m2']}, inplace = True)


In [24]:
dfprop.isnull().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    17992
lat-lon                        48544
lat                            48544
lon                            48544
price                          14973
currency                       14974
price_aprox_local_currency     14973
price_aprox_usd                14973
surface_total_in_m2             7283
surface_covered_in_m2           7283
price_usd_per_m2               45984
price_per_m2                   27407
floor                         104901
rooms                          67543
expenses                       98647
properati_url                      0
description                        2
title                              0
image_thumbnail                 2666
dtype: int64

Imputamos la variable 'price_per_m2' calculandola en base a 'price_aprox_us' y 'surface_total_in_m2' 

In [25]:

dfprop.loc[dfprop['price_usd_per_m2'].isna() &\
           (~dfprop['price_aprox_usd'].isna())&\
            (~dfprop['surface_total_in_m2'].isna())&\
            (dfprop['surface_total_in_m2']!=0.),\
           'price_usd_per_m2']= dfprop['price_aprox_usd']/dfprop['surface_total_in_m2']

In [26]:
dfprop.isnull().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    17992
lat-lon                        48544
lat                            48544
lon                            48544
price                          14973
currency                       14974
price_aprox_local_currency     14973
price_aprox_usd                14973
surface_total_in_m2             7283
surface_covered_in_m2           7283
price_usd_per_m2               22497
price_per_m2                   27407
floor                         104901
rooms                          67543
expenses                       98647
properati_url                      0
description                        2
title                              0
image_thumbnail                 2666
dtype: int64

In [27]:
dfprop.shape

(112458, 25)

Eliminamos propiedades sin precio y con cantidad de metro=0

In [28]:

dfprop=dfprop.drop(dfprop.loc[(dfprop['price_usd_per_m2'].isna())&
                                 (dfprop['price'].isna())&
                                 (dfprop['surface_total_in_m2']==0)].index)

dfprop.shape

(112327, 25)

### Variable  'state_name' vs 'place_with_parent_names'
Notamos que la columna **'place_name'** contiene valores no que aportan una clara referencia de la ubicacion geografica del inmueble.

Por ejemplo
- place_name: 'Centro'
- place_with_parent_names: '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|'

Adicionalmente, los valores contenidos por esta columna no tienen el mismo valor referencial, lo cual difculta entender la precision de dicha columna. 

Por ello utilizamos **'place_with_parent_names'** que nos permite definir correctamente la ubicacion geografica del inmueble. 

Por lo tanto, separamos el texto y generaremos una nueva columna quedando el dataset con esta estructura.

- country_name: Pais
- state_name: provincia (salvo buenos aires que indicara la zona)
- district_name: localidad o partido
- city_name: barrio


In [29]:
# Split por el caracter "|"
dfplacename=[sublista.split('|') for sublista in dfprop['place_with_parent_names']]

# De cada una de las listas quitamos el ultimo y el primer caracter, ya que son vacios
# ['', 'Argentina', 'Bs.As. G.B.A. Zona Norte', 'Vicente López', 'Munro', '']
dfplacename=[sublista[1:5] for sublista in dfplacename]


# Se extrae el nombre de distrito o localidad en una nueva variable denominada 'district_name'
dfprop['district_name']=[str.strip(sublista[2]) if str.strip(sublista[2])!='' else np.nan for sublista in dfplacename]


# Se extrae el nombre de la ciudad o barrio en una nueva variable denominada 'city_name'
dfprop['city_name']=[str.strip(sublista[3]) if (len(sublista)>3  and sublista[3]) else np.nan for sublista in dfplacename]

# Finalmente columnas a ser utilizada seran:
# 'country_name'
# 'state_name'
# 'district_name'
#  'city_name'
# A partir de esas hacer las imputaciones

dfprop[['country_name','state_name','district_name','city_name']].sample(10)

,country_name,state_name,district_name,city_name
51631,Argentina,Bs.As. G.B.A. Zona Norte,Pilar,NaN
100702,Argentina,Bs.As. G.B.A. Zona Sur,Almirante Brown,Jose Marmol
100121,Argentina,Santa Fe,Rosario,NaN
51076,Argentina,Misiones,Posadas,NaN
24306,Argentina,Capital Federal,San Cristobal,NaN
90598,Argentina,Córdoba,Córdoba,NaN
13782,Argentina,Bs.As. G.B.A. Zona Oeste,La Matanza,La Tablada
93218,Argentina,Santa Fe,Rosario,NaN
75375,Argentina,Santa Fe,Rosario,NaN
92152,Argentina,Bs.As. G.B.A. Zona Oeste,General Rodríguez,La Cesarina


### Eliminamos inmuebles sin nombre del distrito

Quitamos casos sin district_name ya que no nos permiten brindar un valor de referencia debido a que no tienen una zona claramente indicada   

In [30]:
mask_nodistrict_nocity_silatlon=(dfprop['district_name'].isna()==True)
dfprop=dfprop.loc[~mask_nodistrict_nocity_silatlon]
dfprop.shape

(108003, 27)

### Eliminamos las propiedades que por 'district_name' no lleguen a 30 operaciones

In [31]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Achiras,7,7,7,7,7,7,7,7,7,7,...,1,1,1,7,0,7,7,7,7,0
Adolfo Alsina,8,8,8,8,8,8,8,3,3,3,...,0,0,0,8,0,8,8,8,8,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Agua de Oro,6,6,6,6,6,6,6,2,2,2,...,3,3,0,1,0,6,6,6,6,0
Aguas Verdes,15,15,15,15,15,15,15,15,15,15,...,6,6,1,14,0,15,15,15,15,0
Aldea Brasilera,1,1,1,1,1,1,1,1,1,1,...,1,1,0,1,0,1,1,1,1,0
Alejandro Roca,1,1,1,1,1,1,0,0,0,0,...,1,1,0,1,0,1,1,1,0,0
Almafuerte,7,7,7,7,7,7,7,3,3,3,...,1,1,0,5,0,7,7,7,7,0


In [32]:
#Creamos dataframe con un valor booleano que indica si district tiene menos de 30 propiedades (True) o no (False).
dfprop_districtless30locations=dfprop.groupby('district_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_districtless30locations.rename(columns={'district_name':'district_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_districtless30locations,how='left',on='district_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a district con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(1895, 27)

In [33]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Almagro,1120,1120,1120,1120,1120,1120,1120,920,920,920,...,941,921,149,706,284,1120,1120,1120,1119,0
Almirante Brown,1520,1520,1520,1520,1520,1520,813,527,527,527,...,1070,999,7,156,93,1520,1520,1520,1454,1481
Avellaneda,628,628,628,628,628,628,615,396,396,396,...,581,547,28,292,62,628,628,628,627,404
Bahía Blanca,456,456,456,456,456,456,456,436,436,436,...,351,311,0,269,10,456,456,456,454,5
Balcarce,80,80,80,80,80,80,80,76,76,76,...,37,32,0,71,1,80,80,80,80,1
Balvanera,626,626,626,626,626,626,626,558,558,558,...,588,559,141,428,191,626,626,626,624,0
Barracas,410,410,410,410,410,410,410,357,357,357,...,380,361,26,235,56,410,410,410,410,0


Corroboramos que el dataframe 'dfprop_fewlocations' se hayan agregado las propiedaes excluidas 

Corroboramos la cantidad de nan en la variable 'price_usd_per_m2'

In [34]:
dfprop.isna().sum()

operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   17696
lat-lon                       44784
lat                           44784
lon                           44784
price                         13563
currency                      13563
price_aprox_local_currency    13563
price_aprox_usd               13563
surface_total_in_m2            6506
surface_covered_in_m2          6506
price_usd_per_m2              20301
price_per_m2                  24886
floor                         98697
rooms                         62404
expenses                      92935
properati_url                     0
description                       2
title                             0
image_thumbnail                2337
district_name                     0
city_name                     68162
dtype: int64

Habiendo eliminado 'district_names' con menos de 30 operaciones, procedemos a quitar aquellos registros intra-district, donde los NaN alcancen o superen el 50% de total de operaciones.

In [35]:
# Obtenemos un dataframe con las propiedades sin precio y con district_name 
dfsin_precio=dfprop.loc[dfprop['price_usd_per_m2'].isna() &\
                        (~dfprop['district_name'].isna())].groupby(['state_name','district_name'])[['operation']].count()

# Obtenemos un dataframe con las propiedades con precio y con district_name
dfcon_precio=dfprop.loc[(~dfprop['price_usd_per_m2'].isna()) &\
                        (~dfprop['district_name'].isna())].groupby(['state_name','district_name'])[['operation']].count()

# Unimos ambos dataframe y agrupamos para obtener la cantidad de propiedades sin y con precio para cada district_name
#Agrupamos por state_name y district_name para evitar que distritos de diferentes provincias se agrupen indebidamente 
df_merge=pd.merge(dfsin_precio,dfcon_precio,how='left',on=['state_name','district_name'])

# Renombro columnas a efectos practicos en la interpretacion
df_merge.rename(columns={'operation_x':'sin precio','operation_y':'con precio'},inplace=True)

# Creo una columna para obtener el total de propiedades
df_merge['total']=df_merge['con precio']+df_merge['sin precio']

# Calculo el % de propiedaes sin precio respecto al total de propiedades
df_merge['%']=df_merge['sin precio']/df_merge['total']

# Filtramos los distritos cuya relacion con la cantidad total de propiedades sea mayor a 0.5
df_merge=df_merge.loc[df_merge['%']>0.5]

#Hacemos un merge con dfprop para que las propiedades a ser removidas queden marcadas
dfpropdelete=pd.merge(dfprop,df_merge,how='left',on=['state_name','district_name'])

# Filtro el dataframe para obtener las propiedades a ingresar en dfprop 
dfprop= dfpropdelete.loc[dfpropdelete['total'].isna()|(~dfpropdelete['total'].isna())&(~dfpropdelete['price_usd_per_m2'].isna())]

# eliminon las columnas utilizadas en este analisis
dfprop.drop(['sin precio','con precio','%','total'],axis=1,inplace=True)

# Filtro el dataframe para obtener las propiedades excluidas 
dfpropdelete= dfpropdelete.loc[(~(dfpropdelete['total'].isna()|(~dfpropdelete['total'].isna())&(~dfpropdelete['price_usd_per_m2'].isna())))]

# elimino las columnas utilizadas en este analisis
dfpropdelete.drop(['sin precio','con precio','%','total'],axis=1,inplace=True)

# Agrego las propiedades excluidas al dataframe 
dfprop_fewlocations=dfprop_fewlocations.append(dfpropdelete)

dfprop.shape

/opt/conda/envs/geopandas/lib/python3.7/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(103889, 27)

In [36]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Almagro,1120,1120,1120,1120,1120,1120,1120,920,920,920,...,941,921,149,706,284,1120,1120,1120,1119,0
Almirante Brown,1520,1520,1520,1520,1520,1520,813,527,527,527,...,1070,999,7,156,93,1520,1520,1520,1454,1481
Avellaneda,628,628,628,628,628,628,615,396,396,396,...,581,547,28,292,62,628,628,628,627,404
Bahía Blanca,456,456,456,456,456,456,456,436,436,436,...,351,311,0,269,10,456,456,456,454,5
Balcarce,37,37,37,37,37,37,37,33,33,33,...,37,32,0,29,1,37,37,37,37,1
Balvanera,626,626,626,626,626,626,626,558,558,558,...,588,559,141,428,191,626,626,626,624,0
Barracas,410,410,410,410,410,410,410,357,357,357,...,380,361,26,235,56,410,410,410,410,0


Corroboramos que el dataframe 'dfprop_fewlocations' se hayan agregado las propiedaes excluidas 

In [37]:
dfprop_fewlocations.shape

(4114, 27)

Corroboramos la cantidad de nan en la variable 'price_usd_per_m2'

In [38]:
dfprop.isna().sum()

operation                         0
property_type                     0
place_name                        0
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                   17370
lat-lon                       43845
lat                           43845
lon                           43845
price                         11550
currency                      11550
price_aprox_local_currency    11550
price_aprox_usd               11550
surface_total_in_m2            6300
surface_covered_in_m2          6300
price_usd_per_m2              18082
price_per_m2                  22667
floor                         96523
rooms                         61168
expenses                      90813
properati_url                     0
description                       2
title                             0
image_thumbnail                2295
district_name                     0
city_name                     66678
dtype: int64

In [39]:
dfprop.groupby(by='state_name').count()

,operation,property_type,place_name,place_with_parent_names,country_name,geonames_id,lat-lon,lat,lon,price,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
state_name,,,,,,,,,,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,24573,24573,24573,24573,24573,14602,11188,11188,11188,23326,...,21868,1772,6870,2980,24573,24572,24573,24275,24573,17211
Bs.As. G.B.A. Zona Oeste,7914,7914,7914,7914,7914,6910,4484,4484,4484,6967,...,6061,557,4043,608,7914,7914,7914,7637,7914,6321
Bs.As. G.B.A. Zona Sur,12591,12591,12591,12591,12591,8461,5214,5214,5214,11504,...,9513,235,2428,1725,12591,12591,12591,12201,12591,10532
Buenos Aires Costa Atlántica,9232,9232,9232,9232,9232,8228,7600,7600,7600,8600,...,6608,147,7350,416,9232,9232,9232,9159,9232,1128
Buenos Aires Interior,1534,1534,1534,1534,1534,1482,958,958,958,1410,...,1131,8,626,64,1534,1534,1534,1519,1534,650
Capital Federal,29555,29555,29555,29555,29555,28346,21897,21897,21897,26921,...,25438,2924,15132,6249,29555,29555,29555,29108,29555,1156
Chubut,109,109,109,109,109,109,0,0,0,106,...,27,0,1,0,109,109,109,76,109,0
Corrientes,530,530,530,530,530,530,2,2,2,448,...,403,0,1,65,530,530,530,528,530,0
Córdoba,7826,7826,7826,7826,7826,7826,3896,3896,3896,6298,...,4459,1209,3230,564,7826,7825,7826,7539,7826,0


In [40]:
#Creamos dataframe con un valor booleano que indica si la provincia tiene menos de 30 propiedades (True) o no (False).
dfprop_stateless30locations=dfprop.groupby('state_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_stateless30locations.rename(columns={'state_name':'state_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_stateless30locations,how='left',on='state_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a provincias con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(44, 27)

In [41]:
dfprop.groupby(by='state_name').count()

,operation,property_type,place_name,place_with_parent_names,country_name,geonames_id,lat-lon,lat,lon,price,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
state_name,,,,,,,,,,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,24573,24573,24573,24573,24573,14602,11188,11188,11188,23326,...,21868,1772,6870,2980,24573,24572,24573,24275,24573,17211
Bs.As. G.B.A. Zona Oeste,7914,7914,7914,7914,7914,6910,4484,4484,4484,6967,...,6061,557,4043,608,7914,7914,7914,7637,7914,6321
Bs.As. G.B.A. Zona Sur,12591,12591,12591,12591,12591,8461,5214,5214,5214,11504,...,9513,235,2428,1725,12591,12591,12591,12201,12591,10532
Buenos Aires Costa Atlántica,9232,9232,9232,9232,9232,8228,7600,7600,7600,8600,...,6608,147,7350,416,9232,9232,9232,9159,9232,1128
Buenos Aires Interior,1534,1534,1534,1534,1534,1482,958,958,958,1410,...,1131,8,626,64,1534,1534,1534,1519,1534,650
Capital Federal,29555,29555,29555,29555,29555,28346,21897,21897,21897,26921,...,25438,2924,15132,6249,29555,29555,29555,29108,29555,1156
Chubut,109,109,109,109,109,109,0,0,0,106,...,27,0,1,0,109,109,109,76,109,0
Corrientes,530,530,530,530,530,530,2,2,2,448,...,403,0,1,65,530,530,530,528,530,0
Córdoba,7826,7826,7826,7826,7826,7826,3896,3896,3896,6298,...,4459,1209,3230,564,7826,7825,7826,7539,7826,0


In [42]:
dfprop.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Mataderos,NaN
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,La Plata,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Mataderos,NaN
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Liniers,NaN
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Mar del Plata,Centro


In [43]:
dfprop['description']=dfprop['description'].str.upper()
dfprop['title']=dfprop['title'].str.upper()
dfprop.head()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Mataderos,NaN
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,VENTA DEPTO 2 DORM. A ESTRENAR 7 E/ 36 Y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,La Plata,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Mataderos,NaN
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 AMBIENTES CON PATIO. HAY 3 DEPTOS EN LOTE...,PH 3 AMB. CFTE. RECICLADO,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Liniers,NaN
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Mar del Plata,Centro


In [44]:
dfprop['amb_aux1']=dfprop['description'].str.find('AMB')
dfprop.head(20)

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name,amb_aux1
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Mataderos,NaN,2.0
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...,VENTA DEPTO 2 DORM. A ESTRENAR 7 E/ 36 Y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,La Plata,NaN,-1.0
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,Mataderos,NaN,2.0
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 AMBIENTES CON PATIO. HAY 3 DEPTOS EN LOTE...,PH 3 AMB. CFTE. RECICLADO,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...,Liniers,NaN,5.0
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,Mar del Plata,Centro,-1.0
5,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...,Vicente López,Munro,-1.0
6,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,Belgrano,NaN,14.0
7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,Belgrano,NaN,14.0
8,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,NaN,NaN,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...,Rosario,NaN,-1.0
9,sell,house,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.4200833,-64.1887761",-31.420083,-64.188776,...,NaN,NaN,NaN,http://www.properati.com.ar/15boy_venta_casa_c...,"HERMOSA CASA SIN TERMINAR DE 3 DORMITORIOS, BA...",CASA SIN TERMINAR EN FALDA DE CAÑETE. ...,https://thumbs4.properati.com/3/mQkhIxXpTTbMSc...,Córdoba,NaN,-1.0


In [45]:
dfprop.dtypes

operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
district_name                  object
city_name   

In [46]:
dfprop.groupby('amb_aux1').count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
amb_aux1,,,,,,,,,,,,,,,,,,,,,
-1.0,44542,44542,44542,44542,44542,44542,37366,22735,22735,22735,...,32283,2383,15275,4216,44542,44542,44542,43015,44542,15792
0.0,199,199,199,199,199,199,186,170,170,170,...,140,1,159,12,199,199,199,199,199,25
1.0,43,43,43,43,43,43,39,35,35,35,...,40,1,39,0,43,43,43,43,43,5
2.0,1359,1359,1359,1359,1359,1359,1262,1063,1063,1063,...,1052,150,1011,280,1359,1359,1359,1300,1359,205
3.0,88,88,88,88,88,88,84,69,69,69,...,68,0,64,13,88,88,88,88,88,10
4.0,1047,1047,1047,1047,1047,1047,954,581,581,581,...,834,22,430,194,1047,1047,1047,1022,1047,217
5.0,372,372,372,372,372,372,339,243,243,243,...,320,12,203,41,372,372,372,368,372,130
6.0,78,78,78,78,78,78,64,38,38,38,...,73,0,32,11,78,78,78,78,78,36
7.0,502,502,502,502,502,502,428,346,346,346,...,428,8,233,38,502,502,502,497,502,227


In [47]:
dfprop['amb_aux1']=dfprop.loc[dfprop['amb_aux1'] !=-1].fillna(0)
dfprop['amb_aux1']=dfprop.loc[dfprop['amb_aux1'] !=-1].astype(np.int64)
dfprop['num_aux1']=dfprop.loc[dfprop['amb_aux1'] !=-11].apply(lambda x: dfprop['description'][(dfprop['amb_aux1']-2):dfprop['amb_aux1']], axis=1)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [48]:
dfprop['property_type'].unique()

array(['PH', 'apartment', 'house'], dtype=object)

In [49]:
a=dfprop['description'].str.find('POSO')
ar=a[((a!=-1)&(~(pd.isna(a))))].index
dfprop.iloc[np.array(ar)].loc[:,'description']

2511     ZONA:RESIDENCIAL-ASFALTO-OBRA SANITARIA:CLOACA...
4740     DEPARTAMENTOS DE 1 AMBIENTE DE 28 M² Y DE 2 AM...
4741     DEPARTAMENTOS DE 1 AMBIENTE DE 32 M² Y DE 2 AM...
4743     DEPARTAMENTOS DE 1 AMBIENTE DE 32 M² Y DE 2 AM...
13978    VENTA - DEPARTAMENTO - ARGENTINA, BUENOS AIRES...
25589    CORREDOR RESPONSABLE: MAURO MARVISI - CMCPSI 5...
28804    CORREDOR RESPONSABLE: MAXIMILIANO VAZQUEZ - C....
31182    CHALET PH EN VENTA, CUENTA CON: LIVING COMEDOR...
37183    CORREDOR RESPONSABLE: RAMIRO CAPPELLONI - C.M....
43650    CORREDOR RESPONSABLE: MATIAS CASTELLI - CUCICB...
52908    DEPARTAMENTO 1 DORMITORIO EN ABASTO, ROSARIO.(...
75790    AMPLIO DEPARTAMENTO DE 4 AMBIENTES CON PATIO I...
77977    VENTA DE DEPARTAMENTO 2 AMBIENTES EN TIGRETIGR...
84773    VENTA DE CASA 4 AMBIENTES EN MARIA EUGENIA RES...
95199    HERMOSA PROPIEDAD UBICADA EN ZONA RESIDENCIAL ...
99359    BEHM SERVICIOS INMOBILIARIOS PRESENTA CASA EN ...
Name: description, dtype: object

In [94]:
def new_dummy_from_included_in_column(dfprop_aux,column,word):
    #dfprop_aux=dfprop
    #word='POZO'
    
    #Creamos la expresion regular ..."que el texto empiece con"... word 
    re_word="("+word+"+\w+)"
    
    #Creamos la expresion regular
    re_search=dfprop[column].str.extract(re_word,expand=False)
    
    #re_search[((re_search!=-1)&(~(pd.isna(re_search))))].index
    #Creamos la columna dummy y seteamos todos los rows en 0
    dfprop_aux.loc[:,"dm_"+word]=0
    
    #Extraemos la lista de de indices donde la expresion se cumple
    prop_re_list =np.array(re_search[((re_search!=-1)&(~(pd.isna(re_search))))].index)
    
    #Actualizamos el valor de la columna Dummy a 1 en los casos dentro del indice enviado
    dfprop_aux.loc[dfprop_aux.index.isin(prop_re_list),"dm_"+word]=1
    
    return dfprop_aux

def new_dummy_from_included(dfprop_aux_,word_):
    dfprop_aux_=new_dummy_from_included_in_column(dfprop_aux_,'description',word_)
    dfprop_aux_=new_dummy_from_included_in_column(dfprop_aux_,'title',word_)
    print_dummy_count_rows(dfprop_aux_,'dm_'+word_)
    return dfprop_aux_

def print_dummy_count_rows(dfprop_aux_p,word_p):
    print('Cantidad de rows con '+word_p+": ",dfprop_aux_p.loc[dfprop_aux_p[word_p]==1,'dm_PARRILLA'].count())
    


In [99]:
dfprop=new_dummy_from_included(dfprop,'CONDOMINIO')
dfprop=new_dummy_from_included(dfprop,'CONSTRUCCION')
dfprop=new_dummy_from_included(dfprop,'QUINCHO')
dfprop=new_dummy_from_included(dfprop,'BARRIO CERRADO')
dfprop=new_dummy_from_included(dfprop,'BAULERA')
dfprop=new_dummy_from_included(dfprop,'VIGILANCIA')
dfprop=new_dummy_from_included(dfprop,'OBRA')
dfprop=new_dummy_from_included(dfprop,'PARRILLA')
dfprop=new_dummy_from_included(dfprop,'CERRADO')
dfprop=new_dummy_from_included(dfprop,'MONOAMBIENTE')
dfprop=new_dummy_from_included(dfprop,'ESCUELA')
dfprop=new_dummy_from_included(dfprop,'ESCUELA')
dfprop=new_dummy_from_included(dfprop,'EXPENSA')




Cantidad de rows con dm_CONDOMINIO:  112
Cantidad de rows con dm_CONSTRUCCION:  1
Cantidad de rows con dm_QUINCHO:  1
Cantidad de rows con dm_BARRIO CERRADO:  0
Cantidad de rows con dm_BAULERA:  14
Cantidad de rows con dm_VIGILANCIA:  0
Cantidad de rows con dm_OBRA:  0
Cantidad de rows con dm_PARRILLA:  14
Cantidad de rows con dm_CERRADO:  15
Cantidad de rows con dm_MONOAMBIENTE:  197
Cantidad de rows con dm_ESCUELA:  1
Cantidad de rows con dm_ESCUELA:  1
Cantidad de rows con dm_EXPENSA:  387


In [106]:
#Ajustamos la dummy de expensas con la existencia de una valor en la columna "expenses"
dfprop.loc[(dfprop['dm_EXPENSA']==0)&(~pd.isna(dfprop['expenses'])),'dm_EXPENSA']=1

print_dummy_count_rows(dfprop,'dm_EXPENSA')

Cantidad de rows con dm_EXPENSA:  13406
